In [76]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchinfo import summary
import numpy as np


In [77]:
class NN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NN, self).__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        # self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.hidden(x))
        out = F.sigmoid(self.output(x))
        return out


In [78]:
# Dataset load
batch_size = 1
train_data = dataset.MNIST(root = 'dataset/', train = True, download = True, transform= transforms.ToTensor())
train_loader = DataLoader(dataset= train_data, batch_size= batch_size, shuffle= True)
test_data = dataset.MNIST(root= 'dataset/', train = False, download= True, transform= transforms.ToTensor())
test_loader = DataLoader(dataset= test_data, batch_size= batch_size, shuffle= True)

In [79]:
X_train, y_train = train_data[0]
y_train = torch.tensor(y_train)
X_val, y_val = test_data[0]
y_val = torch.tensor(y_val)

def con(y):
    temp = [0]* 10
    temp[y] = 1
    y = temp
    return torch.tensor(y, dtype = torch.float32)


# y = [0]* 10
# y[y_val] = 1
# y_val = y
print(y_val)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

tensor(7)


C:\Users\Shabdansh\AppData\Local\Temp\ipykernel_20700\3593199495.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train, dtype=torch.float32)
C:\Users\Shabdansh\AppData\Local\Temp\ipykernel_20700\3593199495.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val = torch.tensor(y_val, dtype=torch.float32)


In [80]:
print(X_train.shape)
print(y_train.shape)


torch.Size([1, 28, 28])
torch.Size([])


In [81]:
def initialize_population(population_size, num_genes):
    return np.random.randint(0, 2, size=(population_size, num_genes))

def decode_chromosome(chromosome, learning_rate_range, num_hidden_neurons_range):
    learning_rate = learning_rate_range[0] + int(''.join(map(str, chromosome[:4])), 2) / 15.0 * (learning_rate_range[1] - learning_rate_range[0])
    num_hidden_neurons = num_hidden_neurons_range[0] + int(''.join(map(str, chromosome[4:])), 2) / 15.0 * (num_hidden_neurons_range[1] - num_hidden_neurons_range[0])
    return learning_rate, int(num_hidden_neurons)

def fitness(learning_rate, num_hidden_neurons, X_train, y_train, X_val, y_val):
    model = NN(input_size= 784, hidden_size=num_hidden_neurons, output_size=10)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)

    num_epochs = 1
    count = 0
    for epoch in range(num_epochs):
        for batch_idx, (X_train, y_train) in enumerate(train_loader):
            count += 1
            X_train = X_train.reshape(X_train.shape[0], -1) # (64, 1*28*28)
            y_train = con(y_train)
            # y_train = y_train.reshape(y_train.shape[0], -1)
            # forward pass
            model.train()
            outputs = model(X_train)
            # print(outputs)
            loss = criterion(outputs[0], y_train)
            # print(outputs.shape)
            # backward
            optimizer.zero_grad()
            loss.backward()

            # gradient descent
            optimizer.step()
            if count == 500:
                break

    model.eval()
    count = 0
    with torch.no_grad():
        accuracy = 0
        for batch_idx, (X_val, y_val) in enumerate(test_loader):
            y_val = con(y_val)
            count += 1
            X_val = X_val.reshape(X_val.shape[0], -1)
            outputs = model(X_val)
            _, predicted = torch.max(outputs, 1)
            accuracy += (predicted == y_val).sum().item() / (len(y_val))
            
            if count == 500:
                break
    # print(accuracy/500)
    return accuracy/500


In [82]:
def genetic_algorithm(population_size, num_genes, learning_rate_range, num_hidden_neurons_range, X_train, y_train, X_val, y_val, num_generations):
    population = initialize_population(population_size, num_genes)

    for generation in range(num_generations):
        fitness_scores = []
        for chromosome in population:
            learning_rate, num_hidden_neurons = decode_chromosome(chromosome, learning_rate_range, num_hidden_neurons_range)
            fitness_scores.append(fitness(learning_rate, num_hidden_neurons, X_train, y_train, X_val, y_val))
        print(generation)
        # parents based on fitness
        parents = population[np.argsort(fitness_scores)[-2:]]
        # print(population)


        # Crossover
        crossover_point = np.random.randint(1, num_genes)
        children = []
        for i in range(population_size - 2):
            parent1 = parents[i % 2]
            parent2 = parents[(i + 1) % 2]
            # child = []
            # for j in range(num_genes):
                # child.append([parent1[j], parent2[j]][np.random.randint(0,2)])

            child = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            # child = np.asarray(child)
            children.append(child)

        # Mutation
        mutation_rate = 0.1
        for i in range(population_size - 2):
            if np.random.rand() < mutation_rate:
                mutation_point = np.random.randint(num_genes)
                children[i][mutation_point] = 1 - children[i][mutation_point]

        # Update population
        population = np.vstack((parents, np.array(children)))

    best_chromosome = population[np.argmax(fitness_scores)]
    best_learning_rate, best_num_hidden_neurons = decode_chromosome(best_chromosome, learning_rate_range, num_hidden_neurons_range)

    return best_learning_rate, best_num_hidden_neurons, max(fitness_scores)


In [83]:
learning_rate_range = [0.001, 0.5]
num_hidden_neurons_range = [5, 10]

best_learning_rate, best_num_hidden_neurons, xyz = genetic_algorithm(population_size=30, num_genes=8,
                                                                learning_rate_range=learning_rate_range,
                                                                num_hidden_neurons_range=num_hidden_neurons_range,
                                                                X_train=X_train, y_train=y_train,
                                                                X_val=X_val, y_val=y_val,
                                                                num_generations=30)

print("Best learning rate:", best_learning_rate)
print("Best number of hidden neurons:", best_num_hidden_neurons)
print("Best fitness score:", xyz)


0
1
2
3
4
5
6
7
8
9
Best learning rate: 0.43346666666666667
Best number of hidden neurons: 6
Best fitness score: 0.563400000000001
